In [38]:
%load_ext autoreload
%autoreload 2
from sympy import symbols, init_printing, simplify, solve, factor
from mako.template import Template
from mako.lookup import TemplateLookup
from mako.runtime import Context
from StringIO import StringIO
from fdlib import *
init_printing()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
U,V,W,Txx,Tyy,Tzz,Txy,Tyz,Txz = IndexedBases('U V W Txx Tyy Tzz Txy Tyz Txz')
Buoyancy, Lambda, Miu = IndexedBases('Buoyancy Lambda Miu')
x,y,z,t,h,dt = symbols('x y z t h dt')

In [5]:
accuracy_time = 1 # O(dt^2) accuracy in time domain
accuracy_space = 2 # O(h^4) accuracy in space domain

In [6]:
l = [x,y,z,t]

# first-order time derivatives in velocities and stresses
# O(dt^2) accuracy
DtU = Deriv_half(U,l,3,dt,accuracy_time)[1]
DtV = Deriv_half(V,l,3,dt,accuracy_time)[1]
DtW = Deriv_half(W,l,3,dt,accuracy_time)[1]

DtTxx = Deriv_half(Txx,l,3,dt,accuracy_time)[1]
DtTxy = Deriv_half(Txy,l,3,dt,accuracy_time)[1]
DtTxz = Deriv_half(Txz,l,3,dt,accuracy_time)[1]
DtTyy = Deriv_half(Tyy,l,3,dt,accuracy_time)[1]
DtTyz = Deriv_half(Tyz,l,3,dt,accuracy_time)[1]
DtTzz = Deriv_half(Tzz,l,3,dt,accuracy_time)[1]

# first-order spacial derivatives of velocities and stresses
# O(h^4) accuracy

DxU = Deriv_half(U,l,0,h,accuracy_space)[1]
DyU = Deriv_half(U,l,1,h,accuracy_space)[1]
DzU = Deriv_half(U,l,2,h,accuracy_space)[1]

DxV = Deriv_half(V,l,0,h,accuracy_space)[1]
DyV = Deriv_half(V,l,1,h,accuracy_space)[1]
DzV = Deriv_half(V,l,2,h,accuracy_space)[1]

DxW = Deriv_half(W,l,0,h,accuracy_space)[1]
DyW = Deriv_half(W,l,1,h,accuracy_space)[1]
DzW = Deriv_half(W,l,2,h,accuracy_space)[1]

DxTxx = Deriv_half(Txx,l,0,h,accuracy_space)[1]
DyTyy = Deriv_half(Tyy,l,1,h,accuracy_space)[1]
DzTzz = Deriv_half(Tzz,l,2,h,accuracy_space)[1]

DxTxy = Deriv_half(Txy,l,0,h,accuracy_space)[1]
DyTxy = Deriv_half(Txy,l,1,h,accuracy_space)[1]
DyTyz = Deriv_half(Tyz,l,1,h,accuracy_space)[1]
DzTyz = Deriv_half(Tyz,l,2,h,accuracy_space)[1]
DxTxz = Deriv_half(Txz,l,0,h,accuracy_space)[1]
DzTxz = Deriv_half(Txz,l,2,h,accuracy_space)[1]

In [10]:
DxTxx

In [11]:
DtTxx

In [20]:
# momentum equations
eq1 = Eq(DtU, Buoyancy[x,y,z]*(DxTxx + DyTxy + DzTxz))
eq2 = Eq(DtV, Buoyancy[x,y,z]*(DxTxy + DyTyy + DzTyz))
eq3 = Eq(DtW, Buoyancy[x,y,z]*(DxTxz + DyTyz + DzTzz))

In [21]:
# stress-strain equations
eq4 = Eq(DtTxx, (Lambda[x,y,z] + 2*Miu[x,y,z])*DxU + Lambda[x,y,z]*(DyV+DzW))
eq5 = Eq(DtTyy, (Lambda[x,y,z] + 2*Miu[x,y,z])*DyV + Lambda[x,y,z]*(DxU+DzW))
eq6 = Eq(DtTzz, (Lambda[x,y,z] + 2*Miu[x,y,z])*DzW + Lambda[x,y,z]*(DxU+DyV))
eq7 = Eq(DtTxy, Miu[x,y,z]*(DyU+DxV))
eq8 = Eq(DtTxz, Miu[x,y,z]*(DzU+DxW))
eq9 = Eq(DtTyz, Miu[x,y,z]*(DzV+DyW))

In [36]:
simplify(solve(eq1,U[x,y,z,t+1])[0]-U[x,y,z,t])+U[x,y,z,t]

In [42]:
U_t = print_assignment(eq1, U[x,y,z,t+1], U[x,y,z,t])
V_t = print_assignment(eq2, V[x,y,z,t+1], V[x,y,z,t])
W_t = print_assignment(eq3, W[x,y,z,t+1], W[x,y,z,t])
Txx_t = print_assignment(eq4, Txx[x,y,z,t+1], Txx[x,y,z,t])
Tyy_t = print_assignment(eq5, Tyy[x,y,z,t+1], Tyy[x,y,z,t])
Tzz_t = print_assignment(eq6, Tzz[x,y,z,t+1], Tzz[x,y,z,t])
Txy_t = print_assignment(eq7, Txy[x,y,z,t+1], Txy[x,y,z,t])
Txz_t = print_assignment(eq8, Txz[x,y,z,t+1], Txz[x,y,z,t])
Tyz_t = print_assignment(eq9, Tyz[x,y,z,t+1], Tyz[x,y,z,t])

In [43]:
Txx_t

'Txx[x][y][z][t + 1]=(1.0L/24.0L)*dt*(-27*Lambda[x][y][z]*U[x][y][z][t] + Lambda[x][y][z]*U[x - 1][y][z][t] + 27*Lambda[x][y][z]*U[x + 1][y][z][t] - Lambda[x][y][z]*U[x + 2][y][z][t] - 27*Lambda[x][y][z]*V[x][y][z][t] + Lambda[x][y][z]*V[x][y - 1][z][t] + 27*Lambda[x][y][z]*V[x][y + 1][z][t] - Lambda[x][y][z]*V[x][y + 2][z][t] - 27*Lambda[x][y][z]*W[x][y][z][t] + Lambda[x][y][z]*W[x][y][z - 1][t] + 27*Lambda[x][y][z]*W[x][y][z + 1][t] - Lambda[x][y][z]*W[x][y][z + 2][t] - 54*Miu[x][y][z]*U[x][y][z][t] + 2*Miu[x][y][z]*U[x - 1][y][z][t] + 54*Miu[x][y][z]*U[x + 1][y][z][t] - 2*Miu[x][y][z]*U[x + 2][y][z][t])/h + Txx[x][y][z][t]'